In [130]:
%matplotlib inline
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json


In [131]:
r = requests.get("http://www.the-numbers.com/movie/records/All-Time-Domestic-Box-Office")
soup = BeautifulSoup(r.text, "lxml")

In [132]:
print(soup.prettify())


<!DOCTYPE html>
<html>
 <head>
  <meta content='(PICS-1.1 "http://www.icra.org/ratingsv02.html" l gen true for "http://www.the-numbers.com/" r (cb 1 lz 1 nz 1 oz 1 vz 1) "http://www.rsac.org/ratingsv01.html" l gen true for "http://www.the-numbers.com/" r (n 0 s 0 v 0 l 0))' http-equiv="PICS-Label"/>
  <!--<meta http-equiv="Content-Type" content="text/html; charset=ISO-8859-1" >-->
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="telephone=no" name="format-detection"/>
  <!-- for apple mobile -->
  <meta content="521546213" property="fb:admins"/>
  <meta content="initial-scale=1" name="viewport"/>
  <meta content="The chart below contains figures for the top 100 movies at the domestic box office, defined as the North American movie territory (consisting of the United States, Canada, Puerto Rico and Guam). This table is updated daily to reflect the latest studio reports." name="description"/>
  <meta content="NOODP" name="robots"/>
  <meta content="

In [133]:
allrows = soup.find_all("tr")

In [134]:
allrows

[<tr>
 <th> </th>
 <th>Released</th>
 <th>Title</th>
 <th>Domestic<br/>Box Office</th>
 <th>International<br/>Box Office</th>
 <th>Worldwide<br/>Box Office</th>
 </tr>, <tr class="highlight">
 <td class="data">1</td>
 <td class="data">2015</td>
 <td><b><a href="/movie/Star-Wars-Ep-VII-The-Force-Awakens#tab=summary">Star Wars Ep. VII: The Force Awakens</a></b></td>
 <td class="data">$936,662,225</td>
 <td class="data">$1,122,000,000</td>
 <td class="data">$2,058,662,225</td>
 </tr>, <tr>
 <td class="data">2</td>
 <td class="data">2009</td>
 <td><b><a href="/movie/Avatar#tab=summary">Avatar</a></b></td>
 <td class="data">$760,507,625</td>
 <td class="data">$2,023,411,357</td>
 <td class="data">$2,783,918,982</td>
 </tr>, <tr>
 <td class="data">3</td>
 <td class="data">1997</td>
 <td><b><a href="/movie/Titanic#tab=summary">Titanic</a></b></td>
 <td class="data">$658,672,302</td>
 <td class="data">$1,548,943,366</td>
 <td class="data">$2,207,615,668</td>
 </tr>, <tr>
 <td class="data">4</t

In [135]:
allrows[1]

<tr class="highlight">
<td class="data">1</td>
<td class="data">2015</td>
<td><b><a href="/movie/Star-Wars-Ep-VII-The-Force-Awakens#tab=summary">Star Wars Ep. VII: The Force Awakens</a></b></td>
<td class="data">$936,662,225</td>
<td class="data">$1,122,000,000</td>
<td class="data">$2,058,662,225</td>
</tr>

In [136]:
onerow = allrows[1].find_all('td')

In [137]:
onerow

[<td class="data">1</td>,
 <td class="data">2015</td>,
 <td><b><a href="/movie/Star-Wars-Ep-VII-The-Force-Awakens#tab=summary">Star Wars Ep. VII: The Force Awakens</a></b></td>,
 <td class="data">$936,662,225</td>,
 <td class="data">$1,122,000,000</td>,
 <td class="data">$2,058,662,225</td>]

In [138]:
rawdata = {'domestic':[],'title':[],'year':[]}
allrows = soup.find_all("tr")
for row in allrows[1:100]:
    el = row.find_all('td')
    year = el[1].contents[0]
    
    
    title = row.find('a').string
       
    gross = (el[3].contents[0]).lstrip('$').replace(",", "")

    rawdata['year'].append(year)
    rawdata['title'].append(title)
    rawdata['domestic'].append(gross)

In [139]:
rawdata

{'domestic': ['936662225',
  '760507625',
  '658672302',
  '652198010',
  '623279547',
  '533345358',
  '483597782',
  '474544677',
  '460998007',
  '459005868',
  '448139099',
  '441226247',
  '435110554',
  '424668047',
  '423315812',
  '422780140',
  '415004880',
  '408992272',
  '408063896',
  '408010692',
  '403706375',
  '402111870',
  '400738009',
  '395708305',
  '381011219',
  '380529370',
  '380270577',
  '377845905',
  '373524485',
  '370782930',
  '368065385',
  '363940561',
  '363070709',
  '362020120',
  '356461711',
  '352390543',
  '351032910',
  '350126372',
  '342548984',
  '341268248',
  '337135885',
  '336530303',
  '336045770',
  '334191110',
  '333172112',
  '330360194',
  '330151138',
  '322719944',
  '319246193',
  '318604126',
  '317575550',
  '317023851',
  '315544750',
  '312433331',
  '310676740',
  '309420425',
  '309205079',
  '308363870',
  '306169255',
  '305411224',
  '304360277',
  '303003568',
  '301959197',
  '300531751',
  '296623634',
  '295983305'

In [140]:
df = pd.DataFrame(rawdata)
df.head()

domestic                                 title  year
0  936662225  Star Wars Ep. VII: The Force Awakens  2015
1  760507625                                Avatar  2009
2  658672302                               Titanic  1997
3  652198010                        Jurassic World  2015
4  623279547                          The Avengers  2012

In [141]:
def wrapper(the_row):
    return get_adjusted_price(the_row.year, the_row.domestic)

df['adj_domestic'] = df.apply(wrapper, axis=1)

ValueError: ("could not convert string to float: '959 937 969.35'", 'occurred at index 0')

In [ ]:
df.sort

In [ ]:
adjusted = requests.get("https://www.statbureau.org/calculate-inflation-price-json?country=united-states&start=1954/1/1&end=2016/8/1&amount=0.32")

In [ ]:
adjusted.status_code

In [ ]:
json.loads(adjusted.text)

In [ ]:
import math 
math.nan

In [ ]:
def get_adjusted_price(year, price):
    url = "https://www.statbureau.org/calculate-inflation-price-json?country=united-states&start={}/1/1&end=2016/8/1&amount={}"
    resp=requests.get(url.format(year,price))
    if resp.status_code == 200:
        adj_price = float(resp.json().replace('$','').replace(',',''))
    else:
        adj_price = math.nan
    return adj_price

print(get_adjusted_price(1990,1.00))